In [1]:

# libraries
import os
import subprocess 
import numpy as np
import pandas as pd
from scipy import signal
import ripple_detection
from ripple_detection import filter_ripple_band
from scipy import stats
from tqdm import tqdm
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from scipy import interpolate
from scipy.signal import firwin, lfilter
from multiprocessing import Pool, Queue, Process
import time

/home/acampbell/miniconda3/envs/allensdk/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gamma_filter_path = '/home/acampbell/NeuropixelsLFPOnRamp/PowerBandFilters/swr_detection_script_filters_1500Hz/frank2008_gamma_1500hz_bandpass_filter.npz'
# The path to the theta filter
theta_filter_path = '/home/acampbell/NeuropixelsLFPOnRamp/PowerBandFilters/swr_detection_script_filters_1500Hz/theta_1500hz_bandpass_filter.npz'
# The ripple band threshold

# loading filters (craetes artifacts in first and last ~ 3.5 seconds of recordings, remember to clip these off)
gamma_filter = np.load(gamma_filter_path)
gamma_filter = gamma_filter['arr_0']

theta_filter = np.load(theta_filter_path)
theta_filter = theta_filter['arr_0']
sdk_cache_dir = '/space/scratch/allen_visbehave_data'
# Setting up the ABI Cache (where data is held, what is present or absent)
manifest_path = os.path.join(sdk_cache_dir, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

# we start by calling and filtering our dataframe of the sessions we will be working with
sessions = cache.get_session_table()

/home/acampbell/miniconda3/envs/allensdk/lib/python3.10/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_cache.py:743: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[array(['CA1', 'VISrl', nan, 'PO', 'LP', 'LGd', 'CA3', 'DG', 'VISl', 'PoT',
        'VISp', 'grey', 'VISpm', 'APN', 'MB', 'VISam'], dtype=object)
 array(['TH', 'Eth', 'APN', 'POL', 'LP', 'DG', 'CA1', 'VISpm', nan, 'NOT',
        'MB', 'SUB', 'VISp', 'VISam', 'grey', 'VL', 'CA3', 'CA2', 'VISl',
        'PO', 'VPM', 'LGd', 'VISal', 'VISrl'], dtype=object)
 array(['MB', 'SCig', 'PPT', 'NOT', 'DG', 'CA1', 'VISam', nan, 'LP', 'VIS',
        'ProS', 'VISp', 'LGv', 'CA3', 'VISl', 'APN', 'POL', 'LGd', 'HPF',
        'VISal', 'VISrl'], dtype=object)
 array(['grey', 'VISpm', nan, 'VISp', 'VISl', 'VISal', 'VISrl'],
       dtype=object)
 array(['grey', 'VISmma', nan, 'VISpm', 'VISp', 'VISl', 'VISrl'],
       dtype=object)
 array(['grey', 'VISam', na

In [3]:
# change this to your local 
abi_swrs_dir_path = '/space/scratch/allen_visbehave_swr_data/allen_visbehave_swr_murphylab2024' 